# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd

# Load the dataset
data = pd.read_csv('homepage_actions.csv')

# Display the first few rows of the dataset
data.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [2]:
# Investigating the id column

# Number of unique users
unique_users = data['id'].nunique()

# Number of users who viewed
users_viewed = data[data['action'] == 'view']['id'].nunique()

# Number of users who clicked
users_clicked = data[data['action'] == 'click']['id'].nunique()

# Number of users who viewed but didn't click
users_viewed_not_clicked = data[(data['action'] == 'view') & (~data['id'].isin(data[data['action'] == 'click']['id']))]['id'].nunique()

# Number of users who clicked but didn't view (should ideally be 0)
users_clicked_not_viewed = data[(data['action'] == 'click') & (~data['id'].isin(data[data['action'] == 'view']['id']))]['id'].nunique()

unique_users, users_viewed, users_clicked, users_viewed_not_clicked, users_clicked_not_viewed


(6328, 6328, 1860, 4468, 0)

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [3]:
#Your code here
# Calculating the CTR for both groups

# Control group
control_views = data[(data['group'] == 'control') & (data['action'] == 'view')].shape[0]
control_clicks = data[(data['group'] == 'control') & (data['action'] == 'click')].shape[0]
control_ctr = control_clicks / control_views

# Experiment group
experiment_views = data[(data['group'] == 'experiment') & (data['action'] == 'view')].shape[0]
experiment_clicks = data[(data['group'] == 'experiment') & (data['action'] == 'click')].shape[0]
experiment_ctr = experiment_clicks / experiment_views

control_ctr, experiment_ctr


(0.2797118847539016, 0.3097463284379172)

In [4]:
#Your code here
from statsmodels.stats.proportion import proportions_ztest

# Number of successes (clicks) for control and experiment groups
count = [control_clicks, experiment_clicks]

# Number of observations (views) for control and experiment groups
nobs = [control_views, experiment_views]

# Conducting two-sample proportion z-test
z_stat, p_value = proportions_ztest(count, nobs, alternative='two-sided')

z_stat, p_value

(-2.618563885349469, 0.008830075576595804)

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [5]:
#Your code here
#Expected number of clicks for the experiment group = CTR of control group ×Number of views in experiment group
expected_clicks_experiment = control_ctr * experiment_views
expected_clicks_experiment

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [6]:
#Your code here
import numpy as np

# Step 2: Calculate the standard deviation and the z-score
std_dev = np.sqrt(experiment_views * control_ctr * (1 - control_ctr))
z = (experiment_clicks - expected_clicks_experiment) / std_dev

std_dev, z

(24.568547907005815, 3.6625360854823588)

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [7]:
from scipy.stats import norm

# Step 3: Calculate the p-value for the z-score
p_value_verification = 2 * (1 - norm.cdf(np.abs(z)))

p_value_verification

0.0002497305601389943

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: 

Comparing the results from the two methods:  The p-value from the two-sample proportion z-test was  0.00883 0.00883. The p-value from the verification method is  0.00025 0.00025. While the exact p-values differ, both are significantly less than the common significance level of  0.05 Thus, both methods lead to the same conclusion: rejecting the null hypothesis.

The slight discrepancy in p-values can be attributed to the different methods used to compute them, but the overall outcome remains consistent.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.